In [1]:
import os 
import pandas as pd
path = '/Users/bokie/code/python/CMS/可转债套利/cms/daily_track'
dates = os.listdir(path)
dates.remove('.DS_Store')
dates.sort()
print('dates \n',dates)
daily_csvs = os.listdir(path+f'/{dates[-1]}')
pair_list = [i for i in daily_csvs if len(i.split('@'))==2]
print(dates[-1],'pair_list \n',pair_list)

dates 
 ['20220909', '20220913', '20220914', '20220915', '20220916', '20220919', '20220920', '20220921', '20220922', '20220923', '20220926', '20220927', '20220928', '20220929', '20220930', '20221010', '20221011', '20221012', '20221013', '20221014', '20221017', '20221018', '20221019', '20221020', '20221021', '20221024', '20221025', '20221026', '20221027', '20221028', '20221031', '20221101', '20221102', '20221103', '20221104', '20221107', '20221108', '20221109', '20221110', '20221111', '20221114', '20221115', '20221116', '20221117', '20221118', '20221121']
20221121 pair_list 
 ['123067.SZ@300382.SZ.csv', '128081.SZ@002203.SZ.csv', '113626.SH@603596.SH.csv', '128017.SZ@002597.SZ.csv', '110061.SH@600674.SH.csv', '128111.SZ@002738.SZ.csv', '113534.SH@603876.SH.csv', '123057.SZ@300586.SZ.csv', '123031.SZ@300655.SZ.csv', '113016.SH@601127.SH.csv', '123092.SZ@300332.SZ.csv', '123046.SZ@300587.SZ.csv', '128095.SZ@002812.SZ.csv', '128106.SZ@002840.SZ.csv', '113025.SH@601677.SH.csv', '128029.SZ@0

In [2]:
pair_dic = {} #key为对子名，value为字典，字典的key为日期，value为该对子对应日期的dataframe
for date in dates:
    tmp_daily_csvs = os.listdir(path+f'/{date}')
    for i in tmp_daily_csvs:
            
        if len(i.split('@'))==2:

            if i not in pair_dic:
                pair_dic[i] = {}

            tmp_df = pd.read_csv(path+f'/{date}/{i}')
            # if len(tmp_df)==3:
            pair_dic[i][date] = (tmp_df)
            

In [3]:
print(pair_dic.keys())
print(len(pair_dic.keys()))
print(len(pair_dic['113642.SH@603185.SH.csv']))
pair_dic['113642.SH@603185.SH.csv']['20220909']

dict_keys(['128081.SZ@002203.SZ.csv', '113642.SH@603185.SH.csv', '128017.SZ@002597.SZ.csv', '110061.SH@600674.SH.csv', '113534.SH@603876.SH.csv', '123031.SZ@300655.SZ.csv', '113016.SH@601127.SH.csv', '123046.SZ@300587.SZ.csv', '123027.SZ@300487.SZ.csv', '113025.SH@601677.SH.csv', '113537.SH@603348.SH.csv', '128029.SZ@002078.SZ.csv', '113626.SH@603596.SH.csv', '128111.SZ@002738.SZ.csv', '123092.SZ@300332.SZ.csv', '128095.SZ@002812.SZ.csv', '123067.SZ@300382.SZ.csv', '123057.SZ@300586.SZ.csv', '128106.SZ@002840.SZ.csv'])
19
40


,Unnamed: 0,pair,cash,signal,bond price,stock price,bond hold,stock hold,commission,tax,reverse open,diff,asset value,net value,open/close
0,0,113642.SH@603185.SH,0,NaN,NaN,NaN,0,0,0,0,NaN,NaN,0,0,"0.023,0"


In [4]:
df_dic = {}
for k in pair_dic.keys():
    df_tmp = pd.DataFrame([])    
    df_tmp['date'] =  list(pair_dic[k].keys())
    df_tmp['date'] = pd.to_datetime(df_tmp['date'])

    df_tmp['net value'] = [v['net value'].tolist()[-1] for v in pair_dic[k].values()]
    df_tmp['net value cum'] = df_tmp['net value'].cumsum()

    df_tmp['bond price'] = [v['bond price'].tolist()[-1] for v in pair_dic[k].values()]
    df_tmp['return rate'] = df_tmp['net value']/df_tmp['bond price']
    df_tmp['return rate'] = df_tmp['return rate'].fillna(0) + 1
    df_tmp['return rate cum'] = df_tmp['return rate'].cumprod()

    df_dic[k]=df_tmp

In [5]:
import plotly.graph_objs as go

traces = [go.Scatter(x=v['date'],y=v['return rate cum'],name=k,) for k,v in df_dic.items()]

trace = go.Scatter(
    x=df_tmp['date'],
    y=df_tmp['return rate cum'],
    name='return rate cum',
    # xaxis='x', 
    # yaxis='y2'#标明设置一个不同于trace1的一个坐标轴
)

data = [trace]
data = traces
# https://blog.csdn.net/weixin_58587245/article/details/127315679 template 可选参数
layout = go.Layout({
                #    "template": 'simple_white',
                #    "template": 'presentation',
                #    "template": 'seaborn',
                #    "template": 'ggplot2',
                   "title":{"text": 'return rate cum', "x": 0.5}, 
                   "xaxis":{"title": {"text": None}, "tickformat": ''},
                   "yaxis":{"title": {"text": ""}},
                   "yaxis2": {'anchor': 'x', "overlaying": 'y', "side": 'right'}, #设置坐标轴的格式，一般次坐标轴在右侧
                   "legend":{"title": {"text": ""}, "x": 0.9, "y": 1.1},
                   "width": 1200,
                   "height": 500})
 
fig = go.Figure(data=data, layout=layout)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=1.3
))

fig.show()